general idea: image is 32x32x4. certain features are also 32x32x4.

why not concatenate all features into the image so it becomes 32x32x16, and then run unet on that?

this code runs unet on "that"

only runs on the 32x32x16 though, no means or whatever included.

In [2]:
import numpy as np # linear algebra
import scipy as scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
import cv2
from tqdm import tqdm


Using Theano backend.


In [3]:
with open('x_train_with_features.pkl', 'rb') as in_strm:
    x_train_with_features = dill.load(in_strm)
with open('x_test_with_features.pkl', 'rb') as in_strm:
    x_test_with_features = dill.load(in_strm)

In [7]:
#creates unet block attached to previous_layer
#returns last layer of unet block
def create_unet_block(depth, width, drop_constant, act, previous_layer):
    prev = previous_layer
    toConcatLater = list()
    
    for cur_depth in range(depth):
        if cur_depth != 0:
            prev = MaxPooling2D(pool_size=(2, 2))(prev)
        conv_width = int(width * math.pow(2, cur_depth))
        conv1 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(prev)
        conv2 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(conv1)
        prev = conv2
        toConcatLater.append(conv2)
    prev = UpSampling2D(size=(2, 2))(prev)
    for cur_depth in reversed(range(depth - 1)):
        merged = concatenate([prev, toConcatLater[cur_depth]], axis=3)
        drop = Dropout(drop_constant)(merged)
        conv_width = int(width * math.pow(2, cur_depth))
        conv1 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(drop)
        conv2 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(conv1)
        if cur_depth != 0:
            up = UpSampling2D(size=(2, 2))(conv2)
            prev = up
        else:
            prev = conv2
    return prev

In [5]:
with open('tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
    y_train = all_info[2]
    y_test = all_info[3]
    label_map = all_info[4]
    inv_label_map = all_info[5]

In [13]:
x_train_with_features = np.array(x_train_with_features)

In [15]:
x_test_with_features = np.array(x_test_with_features)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import math

#baseline part
input_layer = Input(shape=(32, 32, 16))
unet_out = create_unet_block(3, 32, 0.4, 'relu', input_layer)
flat = Flatten()(unet_out)
dense = Dense(17, activation='sigmoid')(flat)
model = Model(inputs=[input_layer], outputs=dense)

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                    optimizer='adam',
                    metrics=['accuracy'])
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 32, 32, 16)    0                                            
____________________________________________________________________________________________________
conv2d_22 (Conv2D)               (None, 32, 32, 32)    4640        input_3[0][0]                    
____________________________________________________________________________________________________
conv2d_23 (Conv2D)               (None, 32, 32, 32)    9248        conv2d_22[0][0]                  
____________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)   (None, 16, 16, 32)    0           conv2d_23[0][0]                  
___________________________________________________________________________________________

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 arrays but instead got the following list of 35000 arrays: [array([[[ 0.10741372,  0.03871242, -0.16329664, ..., -0.73718135,
         -0.55875634, -0.43221534],
        [ 0.14332782,  0.07359379, -0.11497075, ..., -0.40422717,
         -0.30920161, -0.415842...

In [16]:
model.fit(x_train_with_features, y_train, batch_size=128, epochs=6, 
          verbose=1, validation_data=(x_test_with_features, y_test))

from sklearn.metrics import fbeta_score

p_valid = model.predict(x_test_with_features, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

Train on 35000 samples, validate on 5479 samples
Epoch 1/6
35000/35000 [==============================] - 1401s - loss: 0.2186 - acc: 0.9140 - val_loss: 0.2034 - val_acc: 0.9205
Epoch 2/6
35000/35000 [==============================] - 1298s - loss: 0.1803 - acc: 0.9301 - val_loss: 0.1651 - val_acc: 0.9344
Epoch 3/6
35000/35000 [==============================] - 1407s - loss: 0.1539 - acc: 0.9401 - val_loss: 0.1498 - val_acc: 0.9435
Epoch 4/6
35000/35000 [==============================] - 1454s - loss: 0.1438 - acc: 0.9441 - val_loss: 0.1394 - val_acc: 0.9464
Epoch 5/6
35000/35000 [==============================] - 1399s - loss: 0.1362 - acc: 0.9473 - val_loss: 0.1385 - val_acc: 0.9470
Epoch 6/6
35000/35000 [==============================] - 1345s - loss: 0.1315 - acc: 0.9491 - val_loss: 0.1362 - val_acc: 0.9491
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]
[[  2.13296816e-01   1.16660516e-03   4.20724064e

In [17]:
model.fit(x_train_with_features, y_train, batch_size=128, epochs=2, 
          verbose=1, validation_data=(x_test_with_features, y_test))

Train on 35000 samples, validate on 5479 samples
Epoch 1/2
35000/35000 [==============================] - 1327s - loss: 0.1258 - acc: 0.9511 - val_loss: 0.1386 - val_acc: 0.9491
Epoch 2/2
35000/35000 [==============================] - 1338s - loss: 0.1207 - acc: 0.9532 - val_loss: 0.1402 - val_acc: 0.9491


In [18]:
p_valid = model.predict(x_test_with_features, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]
[[  2.72783563e-02   5.23704279e-04   3.14613760e-01 ...,   1.08411326e-03
    7.03453720e-02   1.48136625e-02]
 [  3.51570882e-02   7.88678080e-05   1.48668438e-01 ...,   1.29469542e-03
    1.74294561e-01   1.16271451e-02]
 [  3.09319437e-01   1.11434357e-02   8.64519238e-01 ...,   3.88100816e-05
    2.37639919e-01   9.97262355e-03]
 ..., 
 [  1.98206261e-01   9.48150227e-06   7.71269426e-02 ...,   3.72473987e-05
    8.03205788e-01   1.52661279e-01]
 [  7.42206722e-02   6.60356318e-06   1.05064332e-01 ...,   2.88367064e-05
    7.69380331e-01   1.15221426e-01]
 [  7.09415739e-03   2.95255395e-05   2.67154556e-02 ...,   9.93088815e-06
    7.37909377e-02   2.75290608e-06]]
0.8765107444


In [19]:
adam = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                    optimizer=adam,
                    metrics=['accuracy'])

model.fit(x_train_with_features, y_train, batch_size=128, epochs=2, 
          verbose=1, validation_data=(x_test_with_features, y_test))

Train on 35000 samples, validate on 5479 samples
Epoch 1/2
35000/35000 [==============================] - 1408s - loss: 0.1060 - acc: 0.9586 - val_loss: 0.1336 - val_acc: 0.9513
Epoch 2/2
  256/35000 [..............................] - ETA: 1413s - loss: 0.1030 - acc: 0.9591

KeyboardInterrupt: 

In [20]:
p_valid = model.predict(x_test_with_features, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]
[[  2.81535208e-01   1.02644623e-03   2.81588018e-01 ...,   3.16266110e-03
    2.53828108e-01   4.31184024e-02]
 [  6.16640262e-02   1.26331754e-04   1.38362125e-01 ...,   4.45844838e-04
    3.51460457e-01   1.20070055e-02]
 [  2.66053915e-01   1.35895116e-02   9.49950635e-01 ...,   1.20833670e-06
    2.82630652e-01   8.88649933e-03]
 ..., 
 [  5.63259184e-01   1.27693284e-05   7.69040138e-02 ...,   2.23075149e-05
    8.48186374e-01   2.04686031e-01]
 [  2.30361506e-01   9.98765609e-06   8.69231895e-02 ...,   1.27879621e-05
    8.20713460e-01   1.88403681e-01]
 [  1.44078070e-02   1.02999140e-04   3.29743437e-02 ...,   1.23481441e-05
    1.70316517e-01   2.40943828e-05]]
0.882496988265


In [21]:
model.fit(x_train_with_features, y_train, batch_size=128, epochs=3, 
          verbose=1, validation_data=(x_test_with_features, y_test))

Train on 35000 samples, validate on 5479 samples
Epoch 1/3
35000/35000 [==============================] - 1373s - loss: 0.1022 - acc: 0.9601 - val_loss: 0.1349 - val_acc: 0.9509
Epoch 2/3
35000/35000 [==============================] - 1379s - loss: 0.0999 - acc: 0.9610 - val_loss: 0.1357 - val_acc: 0.9506
Epoch 3/3
  512/35000 [..............................] - ETA: 1553s - loss: 0.0946 - acc: 0.9620

KeyboardInterrupt: 

In [22]:
p_valid = model.predict(x_test_with_features, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]
[[  2.98579752e-01   2.03889518e-04   4.83159691e-01 ...,   1.05636474e-03
    3.34187984e-01   2.11036578e-02]
 [  4.36517894e-02   4.11494439e-05   1.50190219e-01 ...,   2.46275595e-04
    3.47909600e-01   1.13198627e-02]
 [  2.39564538e-01   6.95438078e-03   9.83976603e-01 ...,   3.05573337e-07
    3.07481200e-01   9.52917151e-03]
 ..., 
 [  6.64340138e-01   2.40551276e-06   6.79130852e-02 ...,   4.32150318e-06
    8.55286896e-01   1.43277839e-01]
 [  2.19270214e-01   2.38462917e-06   6.84923604e-02 ...,   3.83046017e-06
    8.06023061e-01   1.27953723e-01]
 [  1.36429109e-02   3.33546814e-05   3.23512293e-02 ...,   9.62398190e-06
    1.75585538e-01   3.21469015e-05]]
0.882106962898
